In [5]:
from unified_model.unified import UnifiedModel
from unified_model import mechanical_components as mc
from unified_model import electrical_components as ec
from unified_model.coupling import CouplingModel
from unified_model.governing_equations import unified_ode
from unified_model import parameter_search
from unified_model.utils.utils import collect_samples
import numpy as np

In [4]:
device_A_config = {
    "height": 110 / 1000,
    "magnet_assembly": {
        "m": 1,
        "l_m_mm": 10,
        "l_mcd_mm": 0,
        "dia_magnet_mm": 10,
        "dia_spacer_mm": 10,
    },
    "magnetic_spring": {
        "fea_data_file": "/Users/michael/Nextcloud/nextcloud/PhD/Python/unified_model/data/magnetic-spring/10x10alt.csv",
        "filter_callable": "auto",
        "magnet_assembly": "dep:magnet_assembly",
    },
    "mechanical_spring": {
        "magnet_assembly": "dep:magnet_assembly",
        "strength": 10000000.0,
        "damping_coefficient": None,
    },
    "mechanical_damper": {
        "damping_coefficient": None,
        "magnet_assembly": "dep:magnet_assembly",
    },
    "input_excitation": None,
    "coil_configuration": {
        "c": 1,
        "n_z": 17,
        "n_w": 15,
        "l_ccd_mm": 0,
        "ohm_per_mm": 0.0010789999999999999,
        "tube_wall_thickness_mm": 2,
        "coil_wire_radius_mm": 0.0715,
        "coil_center_mm": 59,
        "inner_tube_radius_mm": 5.5,
        "custom_coil_resistance": 12.5,
    },
    "flux_model": {
        "coil_configuration": "dep:coil_configuration",
        "magnet_assembly": "dep:magnet_assembly",
        "curve_model_path": "../data/flux_curve_model/flux_curve_model_2021_05_11.model",
    },
    "rectification_drop": 0.05,
    "load_model": {"R": 30},
    "coupling_model": {"coupling_constant": None},
    "extra_components": None,
    "governing_equations": {
        "module_path": "unified_model.governing_equations",
        "func_name": "unified_ode",
    },
}

model_A = UnifiedModel.from_config(device_A_config)
model_A.summarize()


  ┌────┐         110
  │    │         105
  │    │         100
  │    │         95
  │    │         90
  │    │         85
  │    │         80
┌─│────│─┐       75
└─│████│─┘       70
  │████│         65
  │    │         60
  │    │         55
  │    │         50
  │    │         45
  │    │         40
  │    │         35
  │    │         30
  │    │         25
  │    │         20
  │    │         15
  │████│         10
  │████│         5
  └────┘         0
🧲 The magnet assembly consists of 1 magnet(s) that are 10mm long and have a diameter of 10mm.
🧲 The magnets' centers are 0mm apart.
🧲 The magnet assembly has a weight of 0.0578N.
🧲 The magnet assembly hovers 46.015mm above the fixed magnet.
-----------
⚡ There are 1 coils, each with 255 windings (17 vertical X 15 horizontal).
⚡ This gives each a coil an estimated height of ~2.43mm and width of ~2.14mm.
⚡ The coils' centers are 0mm apart.
⚡ The first coil's center is 59mm above the fixed magnet.
⚡ The total microgenerator resistance 

In [ ]:
device_B_config = {
    "height": 110 / 1000,
    "magnetic_spring": {
        "fea_data_file": "/Users/michael/Nextcloud/nextcloud/PhD/Python/unified_model/data/magnetic-spring/10x10alt.csv",
        "filter_callable": "auto",
        "magnet_assembly": "dep:magnet_assembly",
    },
    "magnet_assembly": {
        "m": 1,
        "l_m_mm": 10,
        "l_mcd_mm": 0,
        "dia_magnet_mm": 10,
        "dia_spacer_mm": 10,
    },
    "mechanical_spring": {
        "magnet_assembly": "dep:magnet_assembly",
        "strength": 10000000.0,
        "damping_coefficient": None,
    },
    "mechanical_damper": {
        "damping_coefficient": None,
        "magnet_assembly": "dep:magnet_assembly",
    },
    "input_excitation": None,
    "coil_configuration": {
        "c": 1,
        "n_z": 15,
        "n_w": 33,
        "l_ccd_mm": 0,
        "ohm_per_mm": 0.0010789999999999999,
        "tube_wall_thickness_mm": 2,
        "coil_wire_radius_mm": 0.0715,
        "coil_center_mm": 61,
        "inner_tube_radius_mm": 5.5,
        "custom_coil_resistance": 23.5,
    },
    "flux_model": {
        "coil_configuration": "dep:coil_configuration",
        "magnet_assembly": "dep:magnet_assembly",
        "curve_model_path": "../data/flux_curve_model/flux_curve_model_2021_05_11.model",
    },
    "rectification_drop": 0.05,
    "load_model": {"R": 30},
    "coupling_model": {"coupling_constant": None},
    "extra_components": None,
    "governing_equations": {
        "module_path": "unified_model.governing_equations",
        "func_name": "unified_ode",
    },
}

model_B = UnifiedModel.from_config(device_B_config)
model_B.summarize()

In [ ]:
samples_A = collect_samples(
    base_path='../data/2019-05-23/A/',
    acc_pattern='*acc*.csv',
    adc_pattern='*adc*.csv',
    video_label_pattern='*label*.csv'
)

samples_B = collect_samples(
    base_path='../data/2019-05-23/B/',
    acc_pattern='*acc*.csv',
    adc_pattern='*adc*.csv',
    video_label_pattern='*label*.csv'
)

measurements_A = [Measurement(s, model_A) for s in samples_A]
measurements_B = [Measurement(s, model_B) for s in samples_B]

In [ ]:
# Configure the parameter search

import nevergrad as ng

models_and_measurements = [(base_model, measurements)]

instruments = {
    'mech_damping_coefficient': ng.p.Scalar(init=2, lower=0, upper=10),
    'coupling_constant': ng.p.Scalar(init=0, lower=0, upper=10),
    'mech_spring_damping_coefficient': ng.p.Scalar(init=0, lower=0, upper=10),
}

# Run the parameter search
results = parameter_search.mean_of_scores(
    models_and_measurements=models_and_measurements,
    instruments=instruments,
    cost_metric='power',
    budget=10
)

2021-12-06 14:48:08,956	INFO services.py:1263 -- View the Ray dashboard at http://127.0.0.1:8265
